## Load slices of binary data 

In [117]:
import numpy as np
from glob import glob
import pyqtgraph as pq
from fish.image.vol import get_stack_dims, filter_flat, unfilter_flat
%gui qt

In [102]:
# seed correlation
def seed_corr(seed, data):
    from numpy import where, correlate, prod, zeros
    from scipy.stats import zscore
 
    flat = data.reshape(data.shape[0], prod(data.shape[1:])).T
    valid = where(flat.std(1) != 0)[0]
    seed = zscore(seed).copy()
    normed = zscore(flat[valid, :], axis=1)
    
    corrs = zeros(flat.shape[0])
    
    for ind, val in enumerate(valid):
        corrs[val] = correlate(seed, normed[ind]) / len(seed)

    corrs = corrs.reshape(data.shape[1:])
    
    return corrs

# Need this function to prevent pyqtgraph from leaking memory
def wipe_ims(pq):
    from numpy import zeros
    for iw in pq.images:
      iw.setImage(zeros((1,1))) #<- dummy 1x1 image
      iw.parent().close()

In [127]:
base_dir = 'F:/davis/20170501/5dpf_cy302_f1_omr_2_20170501_204050/'
dims = tuple(get_stack_dims(base_dir)[::-1])
fnames = glob(base_dir + '*.stack')
fnames.sort()
print(np.r_[len(fnames), dims])

mem = [np.memmap(f, dtype='uint16', shape=dims) for f in fnames]

[6000   11  912  864]


In [141]:
%%time
wipe_ims(pq)
dat = np.array([m[5,:,:] for m in mem])
im = pq.image(dat)

Wall time: 7.24 s


In [90]:
rois = {}

In [104]:
rois['cere'] = im.roi.getArrayRegion(dat, im.imageItem, axes=(1,2)).mean((1,2))

In [105]:
pq.plot(rois['cere'])

In [132]:
# load ephys
from fish.ephys.ephys import load, windowed_variance
ep_fname = 'W:/davis/data/ephys/20170501/20170501_cy302_f1_omr_2-2.10chFlt'
epdat = load(ep_fname)

In [133]:
from fish.ephys.ephys import estimate_onset

In [134]:
cam = estimate_onset(epdat[2], 3.7, 100)

In [135]:
smth = [windowed_variance(v)[0] for v in epdat[0:2]]

In [136]:
plt.plot(smth[1][cam])
plt.show()

In [140]:
pq.plot(epdat[1][cam[0]:cam[-1]][::10])